In [1]:
import tensorflow as tf
from  tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input,decode_predictions

import PIL
import os
from os import getcwd


plt.style.use('fivethirtyeight')
import pathlib


In [2]:
path_inception = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'



pre_trained_model = InceptionV3(input_shape = (150,150,3),
                               include_top  = False,
                               weights      = 'imagenet')

# Make all the layers in the pre-trained model non-trainable
for layer in pre_trained_model.layers:
      layer.trainable = False

# Print the model summary
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [11]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(pre_trained_model.output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation = 'relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
predictions = layers.Dense  (5, activation = 'softmax')(x)           

model = Model( pre_trained_model.input, outputs = predictions) 

model.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
____________________________________________________________________________________________

In [12]:
daikon = os.path.join('./sorted/Vegetables/Daikon/')
lotus_root = os.path.join('./sorted/Vegetables/lotus root/')
avocado = os.path.join('./sorted/Vegetables/avocado/')
bokchoy = os.path.join('./sorted/Vegetables/bok choy/')
tomato = os.path.join('./sorted/Vegetables/tomato/')
misc = os.path.join('./sorted/Vegetables/misc/')


print('Total Daikon Images:', len(os.listdir(daikon)))
print('Total Lotus Root Images:', len(os.listdir(lotus_root)))
print('Total Avocado Images:', len(os.listdir(avocado)))
print('Total bokchoy Images:', len(os.listdir(bokchoy)))
print('Total tomato Images:', len(os.listdir(tomato)))


Total Daikon Images: 399
Total Lotus Root Images: 408
Total Avocado Images: 476
Total bokchoy Images: 480
Total tomato Images: 350


In [13]:
#Train-Test Partitioning
from glob import glob
import os.path
from shutil import copy2

path = './sorted/Vegetables/'
test = './sorted/Test/'
train = './sorted/Train/'

if not os.path.exists(test):
	os.makedirs(test)
if not os.path.exists(train):
	os.makedirs(train)
 
for folder in glob(path+'/*'):
		print(folder)
		# find number of images in folder
		no_images_in_folder = len(os.listdir(folder))
		# make new folder inside test and train
		folder_test = test+'/'+folder.split('/')[2]+'/'
		folder_train = train+'/'+folder.split('/')[2]+'/'
		# print(folder_test)
		# print(folder_train)
		if not os.path.exists(folder_test):
			os.makedirs(folder_test)
		if not os.path.exists(folder_train):
			os.makedirs(folder_train)

		print("no of images in this folder: {}".format(no_images_in_folder))
		train_num = int(no_images_in_folder*0.8)
		
		# iterate from 0..test and copy to test
		# iterate test to end and copy to train
		for idx, im in enumerate(glob(folder+'/*')):
			#print(im)
			if idx <= train_num:
			# copy to test
				copy2(im, folder_train)
			else:
			# copy to train
				copy2(im, folder_test)



./sorted/Vegetables\avocado
no of images in this folder: 476
./sorted/Vegetables\bok choy
no of images in this folder: 480
./sorted/Vegetables\Daikon
no of images in this folder: 399
./sorted/Vegetables\lotus root
no of images in this folder: 408
./sorted/Vegetables\tomato
no of images in this folder: 350


In [14]:

# Define our example directories and files
train_dir = './sorted/Train/Vegetables/'
validation_dir = './sorted/Test/Vegetables/'

train_daikon_dir = os.path.join(train_dir,'Daikon')
train_avocado_dir = os.path.join(train_dir,'avocado')
train_bokchoy_dir = os.path.join(train_dir,'bok choy')
train_lotusroot_dir = os.path.join(train_dir,'lotus root')
train_tomato_dir = os.path.join(train_dir,'tomato')


validation_daikon_dir = os.path.join(validation_dir,'Daikon')
validation_avocado_dir = os.path.join(validation_dir,'avocado')
validation_bokchoy_dir = os.path.join(validation_dir,'bok choy')
validation_lotusroot_dir = os.path.join(validation_dir,'lotus root')
validation_tomato_dir = os.path.join(validation_dir,'tomato')


train_daikon_fnames = os.listdir(train_daikon_dir)
train_avocado_fnames = os.listdir(train_avocado_dir)
train_bokchoy_fnames = os.listdir(train_bokchoy_dir)
train_lotusroot_fnames = os.listdir(train_lotusroot_dir)
train_tomato_fnames = os.listdir(train_tomato_dir)

validation_daikon_fnames = os.listdir(validation_daikon_dir)
validation_avocado_fnames = os.listdir(validation_avocado_dir)
validation_bokchoy_fnames = os.listdir(validation_bokchoy_dir)
validation_lotusroot_fnames = os.listdir(validation_lotusroot_dir)
validation_tomato_fnames = os.listdir(validation_tomato_dir)

print(len(train_daikon_fnames))
print(len(train_avocado_fnames))
print(len(train_bokchoy_fnames))
print(len(train_lotusroot_fnames))
print(len(train_tomato_fnames))

print(len(validation_daikon_fnames))
print(len(validation_avocado_fnames))
print(len(validation_bokchoy_fnames))
print(len(validation_lotusroot_fnames))
print(len(validation_tomato_fnames))


360
429
433
368
316
79
95
95
81
69


In [15]:
# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,  # This is the source directory for training images
        target_size=(150,150), 
        batch_size=20,
        class_mode='categorical')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,  # This is the source directory for training images
        target_size=(150,150), 
        batch_size=20,
        class_mode='categorical')


Found 1906 images belonging to 5 classes.
Found 419 images belonging to 5 classes.


In [16]:
class_names = train_generator.class_indices

In [17]:

# Define a Callback class that stops training once accuracy reaches 99.0%
class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.99):
            print("\nReached 99.0% accuracy so cancelling training!")
            self.model.stop_training = True

In [ ]:
# Run this and see how many epochs it should take before the callback
# fires, and stops training at 99% accuracy

callbacks = myCallback()
model.fit_generator(train_generator,
                             validation_data = validation_generator,
                             steps_per_epoch = 10,
                             epochs = 10
                             )

Epoch 1/10
10/10 [==============================] - 47s 5s/step - loss: 3.4756 - acc: 0.7650 - val_loss: 1.3973 - val_acc: 0.8711
Epoch 2/10
10/10 [==============================] - 47s 5s/step - loss: 1.7751 - acc: 0.8350 - val_loss: 1.4315 - val_acc: 0.8807
Epoch 3/10
10/10 [==============================] - 46s 5s/step - loss: 2.0754 - acc: 0.8400 - val_loss: 1.2417 - val_acc: 0.8783
Epoch 4/10
10/10 [==============================] - 47s 5s/step - loss: 1.2452 - acc: 0.8950 - val_loss: 0.7974 - val_acc: 0.9093
Epoch 5/10
10/10 [==============================] - 50s 5s/step - loss: 1.2016 - acc: 0.8500 - val_loss: 0.6616 - val_acc: 0.9093
Epoch 6/10
10/10 [==============================] - 49s 5s/step - loss: 1.8396 - acc: 0.7750 - val_loss: 0.5224 - val_acc: 0.9189
Epoch 7/10
10/10 [==============================] - 48s 5s/step - loss: 1.3238 - acc: 0.8600 - val_loss: 0.6841 - val_acc: 0.8974
Epoch 8/10
10/10 [==============================] - 45s 5s/step - loss: 1.7597 - acc: 0.82

In [ ]:
pic_path = './cat.jpg'

In [ ]:
pic = image.load_img(pic_path, target_size =(150,150))

In [ ]:
plt.imshow(pic)

In [ ]:
pic_array = image.img_to_array(pic)

In [ ]:
pic_batch = np.expand_dims(pic_array, axis = 0)

In [ ]:
pic_preprocessed = preprocess_input(pic_batch)


In [ ]:
prediction = model.predict(pic_preprocessed)

In [ ]:
prediction

In [ ]:
score = np.max(prediction)
score

In [ ]:
index = np.where(prediction == np.max(prediction))
index

In [ ]:
class_names = ['Daikon','avocado','bok choy','lotus root','tomato']

In [ ]:
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[(score)], 100 * np.max(score))
)

In [ ]:
class_names = ['Daikon','avocado','bok choy','lotus root']

In [ ]:
(class_names[np.argmax(score)])